# Benders Decomposition

Otherwise known as row generation.

## Typical example: facility location problem

MIP formulation:

$$
\begin{alignat*}{3}
& Z \quad=\quad
    && \text{minimize}   \quad && \sum_j c_j x_j + \sum_{ij} y_{ij} d_{ij} \\
&   && \text{subject to} \quad && \sum_j x_j \leq k \\
&   &&                         && \sum_j y_{ij} \geq 1 \\
&   &&                         && y_{ij} \leq x_j \\
&   &&                         && x_i \in \lbrace 0, 1 \rbrace \\
&   &&                         && y_{ij} \geq 0
\end{alignat*}
$$

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
n = 100
m = 1000
k = 50

c = np.maximum(np.random.normal(loc=1, size=n), 0.1)
d = np.maximum(np.random.normal(loc=1, size=(m, n)), 0.1)

In [3]:
flp = gp.Model("FLP")

x = flp.addVars(n, obj=c, vtype=GRB.BINARY)
y = flp.addVars(m, n, obj=d, lb=0, ub=1)

flp.addConstr(x.sum() <= k)
flp.addConstrs(y.sum(i, "*") >= 1 for i in range(m))
flp.addConstrs(y[i, j] <= x[j] for i in range(m) for j in range(n))

flp

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03


<gurobi.Model Continuous instance FLP: 0 constrs, 0 vars, No parameter changes>

In [4]:
# flp.optimize()

In [5]:
x.values()

[<gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Model Update*>,
 <gurobi.Var *Awaiting Mo

In Benders decomposition we separate the assignment problem from the facility selection problem. Set the so-called recourse function to be:

$$
\begin{alignat*}{4}
& Q(x) \quad=\quad
    && \text{minimize}   \quad && \sum_{ij} y_{ij} d_{ij} && \\
&   && \text{subject to} \quad && \sum_j y_{ij} \geq 1    && \quad [ \alpha_i ] \\
&   &&                         && y_{ij} \leq x_j         && \quad [ \beta_{ij} ] \\
&   &&                         && y_{ij} \geq 0           &&
\end{alignat*}
$$

So that the facility location problem becomes:

$$
\begin{alignat*}{3}
& Z \quad=\quad
    && \text{minimize}   \quad && \sum_j c_j x_j + \theta \\
&   && \text{subject to} \quad && \theta \geq Q(x) \\
&   &&                         && \sum_j x_j \leq k \\
&   &&                         && x_i \in \lbrace 0, 1 \rbrace
\end{alignat*}
$$

The dual of the recourse function problem features $x$ in the objective, but not in the constraints. Then the solutions of the dual lie on the finite set of extreme points of this polyhedron, whatever the $x$. A consequence of this fact is that there exists a linear formulation of the problem above, where the constraint $\theta \geq Q(x)$ is expanded into $V$ contraints, where $V$ is the set of extreme points of the feasibility polyhedron of the problem below:

$$
\begin{alignat*}{3}
& Q(x) \quad=\quad
    && \text{maximize}   \quad && \sum_i \alpha_i - \sum_{ij} \beta_{ij} x_j \\
&   && \text{subject to} \quad && \alpha_i - \beta_{ij} \leq d_{ij} \\
&   &&                         && \alpha_i, \beta_{ij} \geq 0
\end{alignat*}
$$

$V$ is a very large set, so we add constraints lazily by solving for $x^*$ in the master problem, and eliminating $x^*$ if it is infeasible by computing $Q(x^*)$ (thus obtaining $\alpha(x^*), \beta(x^*)$ and adding the constraint $\theta \geq \sum_i \alpha_i(x^*) - \sum_{ij} \beta_{ij}(x^*) x_j$ to the master problem.

In [6]:
master = gp.Model("master")

x = master.addVars(n, obj=c, vtype=GRB.BINARY)
t = master.addVar(lb=0, obj=1)

master.addConstr(x.sum() <= k)

master

<gurobi.Model Continuous instance master: 0 constrs, 0 vars, No parameter changes>

In [7]:
sub = gp.Model("sub")

sub.modelsense = GRB.MAXIMIZE
alpha = sub.addVars(m, obj=1.0, lb=0)
beta = sub.addVars(m, n, obj=0, lb=0)

sub.addConstrs(alpha[i] - beta[i, j] <= d[i, j] for i in range(m) for j in range(n))

sub

<gurobi.Model Continuous instance sub: 0 constrs, 0 vars, No parameter changes>

In [ ]:
ub = float("inf")
lb = float("-inf")

while ub - lb > lb * 1e-4:
    master.params.outputflag = 0
    master.optimize()

    for (i, j), var in beta.items():
        var.obj = - x[j].x

    sub.params.outputflag = 0
    sub.params.infunbdinfo = 1
    sub.optimize()
    
    lb = master.getObjective().getValue()
    ub = lb - t.x + sub.getObjective().getValue()
    
    if lb > 0:
        print(f"Gap: {ub / lb - 1:2.2f}%\tBound: {ub:3.2f}")

    if sub.status == GRB.UNBOUNDED:
        expr = (
            gp.quicksum(a.unbdray for a in alpha.values())
            + gp.quicksum(- beta[i, j].unbdray * x[j] for i in range(m) for j in range(n)))
        master.addConstr(expr <= 0)

    elif sub.status == GRB.OPTIMAL:
        expr = (
            gp.quicksum(a.x for a in alpha.values())
            + gp.quicksum(- beta[i, j].x * x[j] for i in range(m) for j in range(n)))
        master.addConstr(t >= expr)